In [1]:
!pip install qiskit
!pip install pylatexenc
!pip install qiskit.ignis
!pip install qiskit_machine_learning
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 13.1 MB/s eta 0:00:00


In [2]:
from qiskit import *
# Qiskit module
from qiskit import QuantumCircuit
import qiskit.circuit.library as circuit_library
import qiskit.quantum_info as qi
#from qiskit import execute
from qiskit.utils import algorithm_globals
from qiskit.circuit.library import EfficientSU2
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
import torch
from qiskit.circuit import ParameterVector
from torch import Tensor
import torch.nn as nn
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from tqdm.notebook import tqdm

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [4]:
# path='/content/gdrive/MyDrive/New_test/'

# Preprocessing using some vectorization codes
- We can use the Data2Vec model ([paper](https://arxiv.org/abs/2202.03555)) which can be loaded from [Hugging Face Library](https://huggingface.co/docs/transformers/model_doc/data2vec#transformers.Data2VecVisionModel)
- It provides vectorization of text, image and audio and has many models available for different use cases.
- Here we first download IMDB and CIFAR datasets from tensorflow and load them as pandas dataframes.
- Later we download the pretrained tokenizer, image processor, along with Data2Vec models for both text and images.
- The D2Vset is a custom PyTorch compatible dataset. This means it can be used with PyTorch dataloaders during training. It takes any dataframe containing either a column named text or a column named image along with another column named label and creates a dataset. This dataset is then wrapped with a dataloader which can also help us shuffle and take batches of inputs.

In [5]:
from transformers import AutoTokenizer, AutoImageProcessor
from transformers import Data2VecTextModel, Data2VecTextConfig #as D2Vtxt
from transformers import Data2VecVisionModel, Data2VecVisionConfig #as D2Vimg
from torch.utils.data import DataLoader, Dataset
from tensorflow import compat
import tensorflow_datasets as tfds

In [6]:
# Loading IMDB Dataset from tensorflow datasets
imdb_data = tfds.load("imdb_reviews", shuffle_files=True)
imdb_train_data = tfds.as_dataframe(imdb_data['train'].take(64))
imdb_test_data = tfds.as_dataframe(imdb_data['test'].take(64))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCGJFCR/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCGJFCR/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCGJFCR/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [7]:
# Loading Cifar10 Dataset from tensorflow datasets
cifar_data = tfds.load("cifar10", shuffle_files=True)
cifar_train_data = tfds.as_dataframe(cifar_data['train'].take(64)).drop('id', axis=1)
cifar_test_data = tfds.as_dataframe(cifar_data['test'].take(64)).drop('id', axis=1)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteNT5H8Y/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteNT5H8Y/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [8]:
# Custom PyTorch Dataset for data loading and preprocessing both image and text based on Data2Vec Model
# Input data must be a DataFrame in a particular format. For text one column must be text and other must be label. For images one column should be image and other label.
# The column name for input text must be text and column name for input image must be image and for the ground truth must be label.

class D2Vset(Dataset):
  def __init__(self, data_df, tokenizer=None, txtvecmod=None, imgprocessor=None, imgvecmod=None, max_seq_length=None, padding=True, transform=None, target_transform=None):
    self.df = data_df
    self.tokenizer = tokenizer
    self.txtvectorizer = txtvecmod
    self.imgprocessor = imgprocessor
    self.imgvectorizer = imgvecmod
    self.max_seq_length = max_seq_length
    self.transform = transform
    self.target_transform = target_transform
    self.pad = padding

    if 'text' in self.df.columns:
      self.task = 'txt'
      if (self.tokenizer is None) or (txtvecmod is None):
        raise ValueError("Please reinitialize the Dataset with an AutoTokenizer and a Data2VecTextModel")
    elif 'image' in self.df.columns:
      self.task = 'img'
      if (self.imgprocessor is None) or (imgvecmod is None):
        raise ValueError("Please reinitialize the Dataset with an AutoImageProcessor and a Data2VecVisionModel")
    elif ('text' in self.df.columns) and ('image' in self.df.columns):
      raise KeyError("Both text and image column names cannot be present in the same dataset. Drop either one column and proceed again!")
    else:
      raise KeyError("Either one of text or image must be column names to specify whether it is a text or image task!")

    if self.max_seq_length is not None:
      self.truncation = True
    else:
      self.truncation = False

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, idx):
    data = self.df.iloc[idx]
    label = data['label']
    if self.task == 'txt':
      feature = self._txt2vec(data['text'])
    elif self.task == 'img':
      feature = self._img2vec(data['image'])
    else:
      raise ValueError("Please provide a valid task and ensure it is defined!")

    return feature.squeeze(), label.squeeze()

  def _img2vec(self, img):
    vectorizer_inputs = self.imgprocessor(img, return_tensors='pt')
    with torch.no_grad():
      img_vec = self.imgvectorizer(**vectorizer_inputs).last_hidden_state
    return img_vec

  def _txt2vec(self, txt):
    if isinstance(txt, bytes):
        txt = compat.as_str_any(txt)
    vectorizer_inputs = self.tokenizer(txt, return_tensors='pt', truncation=self.truncation, padding=self.pad, max_length=self.max_seq_length)
    with torch.no_grad():
      txt_vec = self.txtvectorizer(**vectorizer_inputs).last_hidden_state
    return txt_vec

In [9]:
# Defining configuration of the pretrained Data2Vec models to be used. This will affect the output size of the tensors (probably last size in the shape)
# text_config = Data2VecTextConfig(hidden_size=6, num_hidden_layers=25, num_attention_heads=3)
# img_config = Data2VecVisionConfig(hidden_size=6, num_hidden_layers=25, num_attention_heads=3)

In [10]:
# Downloading tokenizer, imgprocessor and data2vec models from hugging face
tokenizer = AutoTokenizer.from_pretrained("facebook/data2vec-text-base")
txtvecmod = Data2VecTextModel.from_pretrained("facebook/data2vec-text-base")# config=text_config)#hidden_size=6, num_attention_heads=3, ignore_mismatched_sizes=True)
imgprocessor = AutoImageProcessor.from_pretrained("facebook/data2vec-vision-base")
imgvecmod = Data2VecVisionModel.from_pretrained("facebook/data2vec-vision-base") #config=img_config)#hidden_size=6, num_attention_heads=3, ignore_mismatched_sizes=True)

Some weights of the model checkpoint at facebook/data2vec-text-base were not used when initializing Data2VecTextModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing Data2VecTextModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecTextModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Data2VecTextModel were not initialized from the model checkpoint at facebook/data2vec-text-base and are newly initialized: ['data2vec_text.pooler.dense.weight', 'data2vec_text.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.10/dist-packages/transformers/models/beit/image_processing_beit.py:108: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [11]:
# Text processing with little bit of imdb data
imdb_trainset = D2Vset(data_df=imdb_train_data, tokenizer=tokenizer, txtvecmod=txtvecmod, imgprocessor=None, imgvecmod=None, max_seq_length=197, padding='max_length', transform=None, target_transform=None)
imdb_trainloader = DataLoader(imdb_trainset, shuffle=True, batch_size=32)
imdb_feature, imdb_label = next(iter(imdb_trainloader))
print(imdb_feature.shape, imdb_label.shape)

torch.Size([32, 197, 768]) torch.Size([32])


In [12]:
# Image processing with little bit of CIFAR data
cifar_trainset = D2Vset(data_df=cifar_train_data, tokenizer=None, txtvecmod=None, imgprocessor=imgprocessor, imgvecmod=imgvecmod, transform=None, target_transform=None)
cifar_trainloader = DataLoader(cifar_trainset, shuffle=True, batch_size=32)
cifar_feature, cifar_label = next(iter(cifar_trainloader))
print(cifar_feature.shape, cifar_label.shape)

torch.Size([32, 197, 768]) torch.Size([32])


# Modifying I Chi's code to classify with the preprocessor
- Since the preprocessor gives a large tensor as output and we would need to train a large model for getting the desired tensor sizes, we can introduce a small fully connected network which performs this size conversion for us.
- This is the `self.initial_layer` in the code. The code now takes three more parameters as input while instantiating the model. The parameters as defined as follows.
- `vec_inp_size:` This is the size of the tensor obtained from the dataset. This tensor comes from the vectorizers and we can see the shapes for the image and text features being printed above. These are the shapes to be given to this parameter while instantiating.
- `mod_inp_size:` This is the input size requirement of I Chi's Model. If (32, 5, 6) is the size requirement then it should be fed as it is. Both these sizes must be provided as tuples.
- `batch_size:` This is the batch size we use when initializing the dataloaders. It is currently 32.
- `init_hid_size:` This is the size of the hidden layer to use for the size conversion in the fully connected layer. It is currently 1000 but can be arbitrary.
- We have also defined a `model_trainer` function which takes an instantiated model, number of epochs and the training dataloader as inputs and returns a trained model as the output. The subsequent cells show separate models being trained for a single epoch on the imdb dataloader and the cifar dataloader

In [13]:
# Modified I Chi's code
import random
class QSAL_qiskit(torch.nn.Module):
    def __init__(self,S,n,Denc,D):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.seq_num=S
        self.init_params_Q=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.init_params_K=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.init_params_V=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.params_input=[ParameterVector('IN'+str(i),n*(Denc+2)) for i in range(self.seq_num)]
        self.params_Q=[ParameterVector('Q'+str(i),n*(D+2)) for i in range(self.seq_num)]
        self.params_K=[ParameterVector('K'+str(i),n*(D+2)) for i in range(self.seq_num)]
        self.params_V=[ParameterVector('V'+str(i),n*(D+2)) for i in range(self.seq_num)]
        self.num_q=n
        self.Denc=Denc
        self.D=D
        self.d=n*(Denc+2)
        self.bit_string_Z=SparsePauliOp.from_list([('I'*(self.num_q-1)+'Z', 1)])
        self.pauli_strings=[]
        for i in range(self.d):
            string=['I']*self.num_q
            while string==['I']*self.num_q:
                for j in range(self.num_q):
                    a=random.randint(0, 4)
                    if a==0:
                        continue
                    elif a==1:
                        string[j]='X'
                    elif a==2:
                        string[j]='Y'
                    elif a==3:
                        string[j]='Z'
            self.pauli_strings.append(SparsePauliOp.from_list([("".join(string), 1)]))

        Q_qnn=[EstimatorQNN(circuit=self.QSAL_cir("Q",i),observables=[self.bit_string_Z], input_params=self.params_input[i], weight_params=self.params_Q[i]) for i in range(self.seq_num)]
        K_qnn=[EstimatorQNN(circuit=self.QSAL_cir("K",i),observables=[self.bit_string_Z], input_params=self.params_input[i], weight_params=self.params_K[i]) for i in range(self.seq_num)]
        V_qnn=[EstimatorQNN(circuit=self.QSAL_cir("V",i),observables=self.pauli_strings, input_params=self.params_input[i], weight_params=self.params_V[i]) for i in range(self.seq_num)]

        self.Q_models=[TorchConnector(Q_qnn[i], initial_weights=self.init_params_Q[i]) for i in range(self.seq_num)]
        self.K_models=[TorchConnector(K_qnn[i], initial_weights=self.init_params_K[i]) for i in range(self.seq_num)]
        self.V_models=[TorchConnector(V_qnn[i], initial_weights=self.init_params_V[i]) for i in range(self.seq_num)]

    def QSAL_cir(self,type,indx):

        qc=QuantumCircuit(self.num_q)
        if type=="Q":
            self.Feature_map(qc,self.params_input[indx])
            self.ansatz(qc,self.params_Q[indx])

        elif type=="K":
            self.Feature_map(qc,self.params_input[indx])
            self.ansatz(qc,self.params_K[indx])

        elif type=="V":
            self.Feature_map(qc,self.params_input[indx])
            self.ansatz(qc,self.params_V[indx])

        return qc

    def Feature_map(self,qc,params):
        indx=0
        for j in range(self.num_q):
            qc.rx(params[indx],j)
            qc.ry(params[indx+1],j)
            indx+=2
        for i in range(self.Denc):
            for j in range(self.num_q):
                qc.cx(j,(j+1)%self.num_q)

            for j in range(self.num_q):
                #qc.rx(params[indx],j)
                qc.ry(params[indx],j)
                indx+=1


    def ansatz(self,qc,params):
        indx=0
        for j in range(self.num_q):
            qc.rx(params[indx],j)
            qc.ry(params[indx+1],j)
            indx+=2
        for i in range(self.D):
            for j in range(self.num_q):
                qc.cx(j,(j+1)%self.num_q)

            for j in range(self.num_q):
                #qc.rx(params[indx],j)
                qc.ry(params[indx],j)
                indx+=1

    def forward(self,input):

        Q_output=torch.stack([self.Q_models[i](input[:,i]) for i in range(self.seq_num)])
        K_output=torch.stack([self.K_models[i](input[:,i]) for i in range(self.seq_num)])
        V_output=torch.stack([self.V_models[i](input[:,i]) for i in range(self.seq_num)])
        batch_size=len(input)
        Q_output=Q_output.transpose(0,2).repeat((self.seq_num,1,1))
        K_output=K_output.transpose(0,2).repeat((self.seq_num,1,1)).transpose(0,2)
        #print(V_output.size())
        #Q_grid, K_grid=torch.meshgrid(Q_output, K_output, indexing='ij')
        alpha=torch.exp(-(Q_output-K_output)**2)
        alpha=alpha.transpose(0,1)
        V_output=V_output.transpose(0,1)
        output=[]

        for i in range(self.seq_num):

            Sum_a=torch.sum(alpha[:,i,:],-1)
            div_sum_a=(1/Sum_a).repeat(self.d,self.seq_num,1).transpose(0,2)

            Sum_w=torch.sum(alpha[:,:,i].repeat((self.d,1,1)).transpose(0,2).transpose(0,1)*V_output*div_sum_a,1)
            output.append(Sum_w)
        return input+torch.stack(output).transpose(0,1)


class QSANN_qiskit(torch.nn.Module):
    def __init__(self,S,n,Denc,D,num_layers):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.qsal_lst=[QSAL_qiskit(S,n,Denc,D) for _ in range(num_layers)]
        self.qnn=nn.Sequential(*self.qsal_lst)

    def forward(self,input):

        return self.qnn(input)


class QSANN_text_classifier(torch.nn.Module):  # MODIFIED TO ADD A LINEAR LAYER WHICH CAN CONVERT VECTOR SIZES ACCORDING TO NEED
    def __init__(self,S,n,Denc,D,num_layers, vec_inp_size, mod_inp_size, batch_size, init_hid_size):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value

        # vec_inp_size: size of vector obtained from the text or image vectorizers
        # mod_inp_size: input size requirement of I Chi's Model
        # batch_size: batch size of input
        # init_hid_size: hidden layer size of fully connected layer used for size conversion (Can be arbritrary. Suggested 1000)
        """
        super().__init__()
        self.mod_inp_size = mod_inp_size
        self.batch_size = batch_size
        self.initial_layer = nn.Sequential(
            nn.Linear(np.prod(vec_inp_size[1:]), init_hid_size),
            nn.Linear(init_hid_size, np.prod(mod_inp_size[1:])),
            )  # input sizes must be provided as (batch_size x 2nd dim x 3rd dim) (eg. 1 x 5 x 6 )
        self.Qnn=QSANN_qiskit(S,n,Denc,D,num_layers)
        self.final_layer=nn.Linear(n*(Denc+2)*S, 1)
        self.final_layer=self.final_layer.float()

    def forward(self,input):
      x = self.initial_layer(torch.flatten(input, start_dim=1))
      x = x.view((self.batch_size, *self.mod_inp_size[1:]))
      x=self.Qnn(x)
      x=torch.flatten(x,start_dim=1)
      return torch.sigmoid(self.final_layer(x))


def model_trainer(model, n_epochs, trainloader):
  '''
  Model trainer to train QSANN
  Parameters:
    model (PyTorch Model): QSANN model for text or image classification with correct sizes specified
    n_epochs (int): Number of epochs to train for.
    trainloader (PyTorch Dataloader): Dataloader containing the dataset.
  '''
  optimizer = torch.optim.Adam(lr=0.03, params=model.parameters())
  criterion = nn.BCELoss()
  pbar = tqdm(total=len(trainloader), leave=True)
  for epoch in range(n_epochs):
    for batch, (feature, label) in enumerate(trainloader):
        optimizer.zero_grad()
        predictions = model(feature.float()).squeeze(1)
        loss = criterion(predictions, label.float())
        loss.backward()
        optimizer.step()
        pbar.update()
        pbar.desc = f"Epoch: {epoch} | Batch: {batch}"
        #print(acc)
        #print(loss)
  pbar.refresh()
  pbar.close()
  return model

In [14]:
# Training and testing the text model
text_model = QSANN_text_classifier(5,2,1,1,1, (32, 197, 768), (32, 5, 6), 32, 1000)  # Instantiating the model
trained_text_model = model_trainer(text_model, 1, imdb_trainloader)  # Training the model
# trained_text_model(imdb_feature)  # Performing a random prediction

/usr/local/lib/python3.10/dist-packages/qiskit_machine_learning/connectors/torch_connector.py:312: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self._weights.data = torch.tensor(initial_weights, dtype=torch.float)


  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
# Training and testing the image model
image_model = QSANN_text_classifier(5,2,1,1,1, (32, 197, 768), (32, 5, 6), 32, 1000)  # Instantiating the model
trained_image_model = model_trainer(image_model, 1, cifar_trainloader)  # Training the model
# trained_image_model(cifar_feature)  # Performing a random prediction

  0%|          | 0/2 [00:00<?, ?it/s]